## Extração de Caracteristicas

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from keras.applications.vgg16 import VGG16, preprocess_input

def load_img(img_path, target_size):
    img = Image.open(img_path)
    img = img.resize(target_size)
    return img

def extract_features(directory):
    model = VGG16(weights='imagenet', include_top=False)
    features = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            label = directory.split('/')[-1]
            try:
                image = load_img(img_path, target_size=(224, 224))
                image = np.array(image)
                image = np.expand_dims(image, axis=0)
                image = preprocess_input(image)
                feature = model.predict(image).flatten()
                features.append(feature)
                labels.append(label)
            except Exception as e:
                print(f"Error processing image: {img_path}")
    return features, labels

# Treino
cats_dir = "./data/train/cats"
dogs_dir = "./data/train/dogs"

cat_features, cat_labels = extract_features(cats_dir)
dog_features, dog_labels = extract_features(dogs_dir)

all_features = cat_features + dog_features
all_labels = cat_labels + dog_labels

data = {'class': all_labels}
for i in range(len(all_features[0])):
    data[f'feature_{i+1}'] = [feat[i] for feat in all_features]

df = pd.DataFrame(data)

csv_train_path = "./data/train/features.csv"
df.to_csv(csv_train_path, index=False)

# Teste
cats_dir = "./data/test/cats"
dogs_dir = "./data/test/dogs"

cat_features, cat_labels = extract_features(cats_dir)
dog_features, dog_labels = extract_features(dogs_dir)

all_features = cat_features + dog_features
all_labels = cat_labels + dog_labels

data = {'class': all_labels}
for i in range(len(all_features[0])):
    data[f'feature_{i+1}'] = [feat[i] for feat in all_features]

df = pd.DataFrame(data)

csv_test_path = "./data/test/features.csv"
df.to_csv(csv_test_path, index=False)

1/1 [==============================] - 1s 767ms/step
Error processing image: ./data/train/dogs\dog_505.jpg
1/1 [==============================] - 1s 799ms/step


## KNN

In [4]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA O KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_grid = {
    'n_neighbors': np.array([3, 5, 7, 9, 11, 13]),
    'metric': np.array(['euclidean', 'manhattan', 'minkowski']),
    'p': np.array([1, 2, 3, 4, 5])
}


# Criando o modelo
modelo = KNeighborsClassifier()

# Criando os grids
gridKNN = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridKNN.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridKNN.best_score_}")
print(f"Melhor K: {gridKNN.best_estimator_.n_neighbors}")
print(f"Melhor distância: {gridKNN.best_estimator_.metric}")
print(f"Melhor p: {gridKNN.best_estimator_.p}")

Fitting 5 folds for each of 90 candidates, totalling 450 fits


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.54319498 0.58275418        nan        nan        nan 0.62035071
 0.78600386        nan        nan        nan 0.72841055 0.75722329
        nan        na

Melhor acurácia: 0.786003861003861
Melhor K: 5
Melhor distância: minkowski
Melhor p: 2


In [5]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = KNeighborsClassifier(n_neighbors=gridKNN.best_estimator_.n_neighbors, metric=gridKNN.best_estimator_.metric, p=gridKNN.best_estimator_.p)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))


Acurácia: 0.8071428571428572
              precision    recall  f1-score   support

        cats       0.81      0.80      0.81        70
        dogs       0.80      0.81      0.81        70

    accuracy                           0.81       140
   macro avg       0.81      0.81      0.81       140
weighted avg       0.81      0.81      0.81       140



# Naive Bayes

In [6]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA NAIVE BAYES

from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

# Criando o modelo
modelo = GaussianNB()

# Criando os grids
gridNB = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridNB.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridNB.best_score_}")
print(f"Melhor var_smoothing: {gridNB.best_estimator_.var_smoothing}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Melhor acurácia: 0.9226512226512227
Melhor var_smoothing: 0.0657933224657568


In [7]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = GaussianNB(var_smoothing=gridNB.best_estimator_.var_smoothing)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Acurácia: 0.9071428571428571
              precision    recall  f1-score   support

        cats       0.98      0.83      0.90        70
        dogs       0.85      0.99      0.91        70

    accuracy                           0.91       140
   macro avg       0.92      0.91      0.91       140
weighted avg       0.92      0.91      0.91       140



# SVM

In [8]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA O SVM

from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_C = np.array([0.001, 0.01, 0.1, 1, 10, 100])
valores_gamma = np.array([0.001, 0.01, 0.1, 1, 10, 100])
tipos_kernel = ['rbf', 'linear', 'poly', 'sigmoid']

valores_grid = {
    'C': valores_C,
    'gamma': valores_gamma,
    'kernel': tipos_kernel
}

# Criando o modelo
modelo = SVC()

# Criando os grids
gridSVM = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridSVM.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridSVM.best_score_}")
print(f"Melhor C: {gridSVM.best_estimator_.C}")
print(f"Melhor gamma: {gridSVM.best_estimator_.gamma}")
print(f"Melhor kernel: {gridSVM.best_estimator_.kernel}")


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Melhor acurácia: 0.9496621621621621
Melhor C: 0.001
Melhor gamma: 0.001
Melhor kernel: linear


In [9]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = SVC(C=gridSVM.best_estimator_.C, gamma=gridSVM.best_estimator_.gamma, kernel=gridSVM.best_estimator_.kernel)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Acurácia: 0.9285714285714286
              precision    recall  f1-score   support

        cats       0.98      0.87      0.92        70
        dogs       0.88      0.99      0.93        70

    accuracy                           0.93       140
   macro avg       0.93      0.93      0.93       140
weighted avg       0.93      0.93      0.93       140



# Decision Trees

In [10]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA O DECISION TREE

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_grid = {
    'criterion': np.array(['gini', 'entropy']),
    'splitter': np.array(['best', 'random']),
    'max_depth': np.array([10, 50, 100, 200, 500]),
    'min_samples_split': np.array([2, 5, 10, 20]),
    'min_samples_leaf': np.array([1, 2, 5, 10])
}

# Criando o modelo
modelo = DecisionTreeClassifier()

# Criando os grids
gridDT = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridDT.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridDT.best_score_}")
print(f"Melhor criterion: {gridDT.best_estimator_.criterion}")
print(f"Melhor splitter: {gridDT.best_estimator_.splitter}")
print(f"Melhor max_depth: {gridDT.best_estimator_.max_depth}")
print(f"Melhor min_samples_split: {gridDT.best_estimator_.min_samples_split}")
print(f"Melhor min_samples_leaf: {gridDT.best_estimator_.min_samples_leaf}")

Fitting 5 folds for each of 320 candidates, totalling 1600 fits
Melhor acurácia: 0.8795688545688545
Melhor criterion: entropy
Melhor splitter: random
Melhor max_depth: 50
Melhor min_samples_split: 20
Melhor min_samples_leaf: 5


In [11]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = DecisionTreeClassifier(criterion=gridDT.best_estimator_.criterion, splitter=gridDT.best_estimator_.splitter, max_depth=gridDT.best_estimator_.max_depth, min_samples_split=gridDT.best_estimator_.min_samples_split, min_samples_leaf=gridDT.best_estimator_.min_samples_leaf)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Acurácia: 0.7571428571428571
              precision    recall  f1-score   support

        cats       0.79      0.70      0.74        70
        dogs       0.73      0.81      0.77        70

    accuracy                           0.76       140
   macro avg       0.76      0.76      0.76       140
weighted avg       0.76      0.76      0.76       140



# Random Forest

In [12]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA O RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados 
valores_n_estimators = np.array([10, 50, 100, 200, 500])
valores_max_depth = np.array([10, 50, 100, 200, 500])
valores_min_samples_split = np.array([2, 5, 10, 20])
valores_min_samples_leaf = np.array([1, 2, 5, 10])

valores_grid = {
    'n_estimators': valores_n_estimators,
    'max_depth': valores_max_depth,
    'min_samples_split': valores_min_samples_split,
    'min_samples_leaf': valores_min_samples_leaf
}

# Criando o modelo
modelo = RandomForestClassifier()

# Criando os grids
gridRF = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridRF.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridRF.best_score_}")
print(f"Melhor n_estimators: {gridRF.best_estimator_.n_estimators}")
print(f"Melhor max_depth: {gridRF.best_estimator_.max_depth}")
print(f"Melhor min_samples_split: {gridRF.best_estimator_.min_samples_split}")
print(f"Melhor min_samples_leaf: {gridRF.best_estimator_.min_samples_leaf}")


Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Melhor acurácia: 0.9658783783783784
Melhor n_estimators: 500
Melhor max_depth: 200
Melhor min_samples_split: 20
Melhor min_samples_leaf: 1


In [13]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = RandomForestClassifier(n_estimators=gridRF.best_estimator_.n_estimators, max_depth=gridRF.best_estimator_.max_depth, min_samples_split=gridRF.best_estimator_.min_samples_split, min_samples_leaf=gridRF.best_estimator_.min_samples_leaf)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Acurácia: 0.9285714285714286
              precision    recall  f1-score   support

        cats       1.00      0.86      0.92        70
        dogs       0.88      1.00      0.93        70

    accuracy                           0.93       140
   macro avg       0.94      0.93      0.93       140
weighted avg       0.94      0.93      0.93       140



# Extra Trees

In [14]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA O EXTRA TREES

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_n_estimators = np.array([10, 50, 100, 200, 500])
valores_max_depth = np.array([10, 50, 100, 200, 500])
valores_min_samples_split = np.array([2, 5, 10, 20])
valores_min_samples_leaf = np.array([1, 2, 5, 10])

valores_grid = {
    'n_estimators': valores_n_estimators,
    'max_depth': valores_max_depth,
    'min_samples_split': valores_min_samples_split,
    'min_samples_leaf': valores_min_samples_leaf
}

# Criando o modelo
modelo = ExtraTreesClassifier()

# Criando os grids
gridET = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridET.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridET.best_score_}")
print(f"Melhor n_estimators: {gridET.best_estimator_.n_estimators}")
print(f"Melhor max_depth: {gridET.best_estimator_.max_depth}")
print(f"Melhor min_samples_split: {gridET.best_estimator_.min_samples_split}")
print(f"Melhor min_samples_leaf: {gridET.best_estimator_.min_samples_leaf}")

Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Melhor acurácia: 0.9676480051480052
Melhor n_estimators: 200
Melhor max_depth: 200
Melhor min_samples_split: 10
Melhor min_samples_leaf: 1


In [15]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = ExtraTreesClassifier(n_estimators=gridET.best_estimator_.n_estimators, max_depth=gridET.best_estimator_.max_depth, min_samples_split=gridET.best_estimator_.min_samples_split, min_samples_leaf=gridET.best_estimator_.min_samples_leaf)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Acurácia: 0.9285714285714286
              precision    recall  f1-score   support

        cats       1.00      0.86      0.92        70
        dogs       0.88      1.00      0.93        70

    accuracy                           0.93       140
   macro avg       0.94      0.93      0.93       140
weighted avg       0.94      0.93      0.93       140



# Ada Boost

In [16]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA O ADA BOOST

from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_n_estimators = np.array([10, 50, 100, 200, 500])
valores_learning_rate = np.array([0.001, 0.01, 0.1, 1, 10, 100])

valores_grid = {
    'n_estimators': valores_n_estimators,
    'learning_rate': valores_learning_rate
}

# Criando o modelo
modelo = AdaBoostClassifier()

# Criando os grids
gridAB = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridAB.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridAB.best_score_}")
print(f"Melhor n_estimators: {gridAB.best_estimator_.n_estimators}")
print(f"Melhor learning_rate: {gridAB.best_estimator_.learning_rate}")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Melhor acurácia: 0.9442728442728443
Melhor n_estimators: 500
Melhor learning_rate: 1.0


In [17]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = AdaBoostClassifier(n_estimators=gridAB.best_estimator_.n_estimators, learning_rate=gridAB.best_estimator_.learning_rate)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Acurácia: 0.9357142857142857
              precision    recall  f1-score   support

        cats       1.00      0.87      0.93        70
        dogs       0.89      1.00      0.94        70

    accuracy                           0.94       140
   macro avg       0.94      0.94      0.94       140
weighted avg       0.94      0.94      0.94       140



# Redes Neurais

In [2]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA REDES NEURAIS

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_grid = {
    'hidden_layer_sizes': [(10,30,10),(20,)], # 1ª camada com 10 neurônios, 2ª com 30 e 3ª com 10
    'activation': ['tanh', 'relu'], # Função de ativação
    'solver': ['sgd', 'adam'], # Gradiente descendente (sgd) ou adam
    'alpha': [0.0001, 0.05], # Taxa de aprendizado
    'learning_rate': ['constant','adaptive'] # Taxa de aprendizado constante ou adaptativa
} 

# Criando o modelo
modelo = MLPClassifier(max_iter=100)

# Criando os grids
gridRN = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridRN.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridRN.best_score_}")
print(f"Melhor hidden_layer_sizes: {gridRN.best_estimator_.hidden_layer_sizes}")
print(f"Melhor activation: {gridRN.best_estimator_.activation}")
print(f"Melhor solver: {gridRN.best_estimator_.solver}")
print(f"Melhor alpha: {gridRN.best_estimator_.alpha}")
print(f"Melhor learning_rate: {gridRN.best_estimator_.learning_rate}")


Fitting 5 folds for each of 32 candidates, totalling 160 fits
Melhor acurácia: 0.9514317889317889
Melhor hidden_layer_sizes: (10, 30, 10)
Melhor activation: tanh
Melhor solver: sgd
Melhor alpha: 0.05
Melhor learning_rate: adaptive


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [3]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = MLPClassifier(hidden_layer_sizes=gridRN.best_estimator_.hidden_layer_sizes, activation=gridRN.best_estimator_.activation, solver=gridRN.best_estimator_.solver, alpha=gridRN.best_estimator_.alpha, learning_rate=gridRN.best_estimator_.learning_rate, max_iter=100)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Acurácia: 0.9
              precision    recall  f1-score   support

        cats       0.92      0.87      0.90        70
        dogs       0.88      0.93      0.90        70

    accuracy                           0.90       140
   macro avg       0.90      0.90      0.90       140
weighted avg       0.90      0.90      0.90       140



c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
